In [14]:
from sklearn.model_selection import train_test_split
import tensorflow as tf
import pandas as pd
import numpy as np

In [15]:
# Read in the datasets
df = pd.read_csv('data/dataset.csv')
df_severities = pd.read_csv('data/Symptom-severity.csv')
df_precautions = pd.read_csv('data/symptom_precaution.csv')
df_descriptions = pd.read_csv('data/symptom_Description.csv')

In [16]:
# Strip the whitespaces
cols = df.columns
data = df[cols].values.flatten()

tmp = pd.Series(data)
tmp = tmp.str.strip()
tmp = tmp.values.reshape(df.shape)

df = pd.DataFrame(tmp, columns=df.columns)

# Fill NaN values with zeros
df = df.fillna(0)

In [17]:
# Fix spelling errors in dataset
df = df.replace('dischromic _patches', 'dischromic_patches')
df = df.replace('spotting_ urination', 'spotting_urination')
df = df.replace('foul_smell_of urine', 'foul_smell_ofurine')
df = df.replace('Dimorphic hemmorhoids(piles)', 'Dimorphic hemorrhoids(piles)')

df.to_csv('data/dataset.csv')

# Replace symptoms with unique id
for i in range(len(df_severities)):
    df = df.replace(df_severities['Symptom'][i], df_severities.index[i]+1)

In [20]:
# List of diseases and symptoms
diseases = list(df['Disease'].unique())
symptoms = list(df_severities['Symptom'].unique())

data = df.iloc[:,1:].values
# Convert string labels into numerical labels and convert to numpy array
labels = [[diseases.index(df['Disease'].values[i])] for i in range(len(df['Disease'].values))]
labels = np.asarray(labels, dtype=np.int64)

# Split data
x_train, x_test, y_train, y_test = train_test_split(data, labels, shuffle=True, train_size = 0.85)

In [21]:
# Create tensorflow model
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Dense(units=256, activation='relu'))
model.add(tf.keras.layers.Dense(units=256, activation='relu'))
model.add(tf.keras.layers.Dense(units=41, activation='softmax')) 

In [22]:
# Compile model
model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

In [23]:
model.fit(x_train, y_train, batch_size=32, epochs=55, validation_split=0.3)
val_loss, val_acc = model.evaluate(x_test, y_test)
print(val_loss)
print(val_acc)

Epoch 1/55
92/92 [==============================] - 4s 8ms/step - loss: 2.2428 - accuracy: 0.5371 - val_loss: 1.0219 - val_accuracy: 0.7243
Epoch 2/55
92/92 [==============================] - 0s 3ms/step - loss: 0.8359 - accuracy: 0.7530 - val_loss: 0.8087 - val_accuracy: 0.8088
Epoch 3/55
92/92 [==============================] - 0s 3ms/step - loss: 0.6767 - accuracy: 0.8176 - val_loss: 0.3962 - val_accuracy: 0.8733
Epoch 4/55
92/92 [==============================] - 0s 3ms/step - loss: 0.4873 - accuracy: 0.8425 - val_loss: 0.4470 - val_accuracy: 0.8661
Epoch 5/55
92/92 [==============================] - 0s 3ms/step - loss: 0.3518 - accuracy: 0.8992 - val_loss: 0.3676 - val_accuracy: 0.8853
Epoch 6/55
92/92 [==============================] - 0s 3ms/step - loss: 0.4325 - accuracy: 0.8753 - val_loss: 0.5428 - val_accuracy: 0.8502
Epoch 7/55
92/92 [==============================] - 0s 3ms/step - loss: 0.3518 - accuracy: 0.8982 - val_loss: 0.2261 - val_accuracy: 0.9386
Epoch 8/55
92/92 [==

In [10]:
# Save model
model.save('symptom_checker.model')

INFO:tensorflow:Assets written to: symptom_checker.model\assets
